In [31]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [32]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Set up the credentials
client_id = "e7c296b4e357419687d613b13497f940"
client_secret = "4b29f4fb12e04dd8b6853eddfccd6bcc"

# Authenticate
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [33]:
# Example: Getting playlists for a user
username = 'byv1tdsf0wr3gpo2hkjkfd0tk'
playlists = sp.user_playlists(username)

for playlist in playlists['items']:
    print(playlist['name'])


Its time
Classical 2
Easy 2
Idfk2
Covers 2.0
idek
idfk
R&B ish
Classical
Covers
Easy
Old


In [34]:
# Example: Getting audio features for a specific track
track_id = '3n3Ppam7vgaVa1iaRUc9Lp'  # Example track ID
features = sp.audio_features(track_id)
import pandas as pd
pd.DataFrame(features)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.355,0.918,1,-4.36,1,0.0746,0.00119,0,0.0971,0.24,148.114,audio_features,3n3Ppam7vgaVa1iaRUc9Lp,spotify:track:3n3Ppam7vgaVa1iaRUc9Lp,https://api.spotify.com/v1/tracks/3n3Ppam7vgaV...,https://api.spotify.com/v1/audio-analysis/3n3P...,222200,4


In [35]:

# Get album details using the random album ID
album_id = '1A2GTWGtFfWp7KSQTwWOyo'
album = sp.album(album_id)

# Get the number of tracks in the album
number_of_tracks = album['tracks']['total']
print(f"The album 'Abbey Road' has {number_of_tracks} tracks.")


The album 'Abbey Road' has 10 tracks.


In [36]:
# Get the number of tracks in the playlist
number_of_tracks = playlist['tracks']['total']
print(f"The playlist has {number_of_tracks} tracks.")

The playlist has 38 tracks.


In [40]:
# Get album details using the random album ID
album_id = '37i9dQZF1DX4SBhb3fqCJd'  # Example playlist ID for R&B
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Create a DataFrame to store the audio features
df = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items']]
df['track_name'] = track_names

# Reorder columns to have track_name first
df = df[['track_name'] + [col for col in df.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 0
df['genre'] = 0
# Display the styled DataFrame
df

HTTP Error for GET to https://api.spotify.com/v1/albums/37i9dQZF1DX4SBhb3fqCJd with Params: {} returned 404 due to Non existing id: 'spotify:album:37i9dQZF1DX4SBhb3fqCJd'


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/albums/37i9dQZF1DX4SBhb3fqCJd:
 Non existing id: 'spotify:album:37i9dQZF1DX4SBhb3fqCJd', reason: None

In [50]:
# rap 1
album_name = "LONG.LIVE.A$AP"
artist_name = "A$AP Rocky"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap1 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap1['track_name'] = track_names

# Reorder columns to have track_name first
df_rap1 = df_rap1[['track_name'] + [col for col in df_rap1.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap1['genre'] = 1

# Display the DataFrame
df_rap1

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,Long Live A$AP,0.726,0.824,7,-5.011,0,0.3450,0.584000,0.000001,0.1130,0.3820,130.118,audio_features,60Mq4EumvcSXznNugo9Op3,spotify:track:60Mq4EumvcSXznNugo9Op3,https://api.spotify.com/v1/tracks/60Mq4EumvcSX...,https://api.spotify.com/v1/audio-analysis/60Mq...,289587,4,1
1,Goldie,0.697,0.849,10,-4.953,0,0.2480,0.228000,0.000045,0.4930,0.7340,81.956,audio_features,31G9RaSaDOI2NWcpnIp734,spotify:track:31G9RaSaDOI2NWcpnIp734,https://api.spotify.com/v1/tracks/31G9RaSaDOI2...,https://api.spotify.com/v1/audio-analysis/31G9...,192067,4,1
2,PMW (All I Really Need) (feat. ScHoolboy Q),0.705,0.747,1,-4.542,1,0.0483,0.000361,0.000000,0.3110,0.2900,129.972,audio_features,5oeKoJMi9g6GXEbtz1ta0f,spotify:track:5oeKoJMi9g6GXEbtz1ta0f,https://api.spotify.com/v1/tracks/5oeKoJMi9g6G...,https://api.spotify.com/v1/audio-analysis/5oeK...,234773,4,1
3,LVL,0.597,0.427,5,-6.764,0,0.0536,0.180000,0.000013,0.1210,0.0997,120.085,audio_features,787rCZF9i4L1cXGMkdyIk4,spotify:track:787rCZF9i4L1cXGMkdyIk4,https://api.spotify.com/v1/tracks/787rCZF9i4L1...,https://api.spotify.com/v1/audio-analysis/787r...,220133,4,1
4,Hell (feat. Santigold),0.668,0.489,7,-9.130,1,0.1370,0.115000,0.000000,0.1840,0.2580,142.226,audio_features,5Uc07fEUpjjFcLEIleEHkJ,spotify:track:5Uc07fEUpjjFcLEIleEHkJ,https://api.spotify.com/v1/tracks/5Uc07fEUpjjF...,https://api.spotify.com/v1/audio-analysis/5Uc0...,231787,4,1
5,Pain (feat. OverDoz.),0.653,0.835,7,-5.769,1,0.2920,0.174000,0.000000,0.3060,0.5150,135.041,audio_features,6CZpEC9kyyCJWWZKPnviXY,spotify:track:6CZpEC9kyyCJWWZKPnviXY,https://api.spotify.com/v1/tracks/6CZpEC9kyyCJ...,https://api.spotify.com/v1/audio-analysis/6CZp...,233267,4,1
6,"F**kin' Problems (feat. Drake, 2 Chainz & Kend...",0.853,0.693,1,-6.870,1,0.2750,0.023900,0.000000,0.1100,0.6620,95.967,audio_features,4X5f3vT8MRuXF68pfjNte5,spotify:track:4X5f3vT8MRuXF68pfjNte5,https://api.spotify.com/v1/tracks/4X5f3vT8MRuX...,https://api.spotify.com/v1/audio-analysis/4X5f...,233787,4,1
7,Wild for the Night (feat. Skrillex & Birdy Nam...,0.757,0.848,2,-4.234,1,0.3350,0.048100,0.000000,0.1210,0.6860,139.995,audio_features,2rzBvHM9h36Tpdj7Jdajka,spotify:track:2rzBvHM9h36Tpdj7Jdajka,https://api.spotify.com/v1/tracks/2rzBvHM9h36T...,https://api.spotify.com/v1/audio-analysis/2rzB...,212640,4,1
8,"1Train (feat. Kendrick Lamar, Joey Bada$$, Yel...",0.489,0.878,2,-3.360,1,0.3680,0.351000,0.000000,0.6980,0.7750,81.497,audio_features,1eLU27PPFxBhbp1CBjPh9J,spotify:track:1eLU27PPFxBhbp1CBjPh9J,https://api.spotify.com/v1/tracks/1eLU27PPFxBh...,https://api.spotify.com/v1/audio-analysis/1eLU...,372173,4,1
9,Fashion Killa,0.801,0.819,11,-5.189,1,0.1580,0.292000,0.000000,0.5150,0.8110,139.969,audio_features,0O3TAouZE4vL9dM5SyxgvH,spotify:track:0O3TAouZE4vL9dM5SyxgvH,https://api.spotify.com/v1/tracks/0O3TAouZE4vL...,https://api.spotify.com/v1/audio-analysis/0O3T...,236280,4,1


In [51]:
# rap 2
album_name = "American Dream"
artist_name = "21 Savage"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap2 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap2['track_name'] = track_names

# Reorder columns to have track_name first
df_rap2 = df_rap2[['track_name'] + [col for col in df_rap2.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap2['genre'] = 1

# Display the DataFrame
df_rap2

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,american dream,0.517,0.367,10,-12.639,0,0.2340,0.504000,0.000405,0.600,0.363,69.073,audio_features,4yreExU3eRNTe2iJz6X6k3,spotify:track:4yreExU3eRNTe2iJz6X6k3,https://api.spotify.com/v1/tracks/4yreExU3eRNT...,https://api.spotify.com/v1/audio-analysis/4yre...,63416,4,1
1,all of me,0.652,0.806,5,-5.707,0,0.3020,0.122000,0.000011,0.842,0.779,159.947,audio_features,2FoahzOSxJnalPA8aBUme3,spotify:track:2FoahzOSxJnalPA8aBUme3,https://api.spotify.com/v1/tracks/2FoahzOSxJna...,https://api.spotify.com/v1/audio-analysis/2Foa...,198293,4,1
2,redrum,0.625,0.733,2,-8.757,1,0.0488,0.005980,0.000983,0.375,0.244,172.090,audio_features,52eIcoLUM25zbQupAZYoFh,spotify:track:52eIcoLUM25zbQupAZYoFh,https://api.spotify.com/v1/tracks/52eIcoLUM25z...,https://api.spotify.com/v1/audio-analysis/52eI...,270698,4,1
3,n.h.i.e.,0.818,0.512,5,-9.056,0,0.0884,0.096300,0.037100,0.110,0.153,131.974,audio_features,6x9pCndnXEoea0CMcfjs9W,spotify:track:6x9pCndnXEoea0CMcfjs9W,https://api.spotify.com/v1/tracks/6x9pCndnXEoe...,https://api.spotify.com/v1/audio-analysis/6x9p...,143719,4,1
4,sneaky,0.742,0.685,9,-7.526,1,0.1490,0.000383,0.004280,0.147,0.306,155.037,audio_features,2K7sv1mno9PvbXVBpqvSKn,spotify:track:2K7sv1mno9PvbXVBpqvSKn,https://api.spotify.com/v1/tracks/2K7sv1mno9Pv...,https://api.spotify.com/v1/audio-analysis/2K7s...,201672,4,1
5,pop ur shit,0.853,0.533,4,-6.647,1,0.1260,0.263000,0.004060,0.134,0.306,123.066,audio_features,47yD0e9MCRIB8dgUXPfyW3,spotify:track:47yD0e9MCRIB8dgUXPfyW3,https://api.spotify.com/v1/tracks/47yD0e9MCRIB...,https://api.spotify.com/v1/audio-analysis/47yD...,193320,4,1
6,letter to my brudda,0.450,0.870,7,-4.224,0,0.3090,0.222000,0.000003,0.291,0.588,156.727,audio_features,5P1z8XwsCStb74emERTx9v,spotify:track:5P1z8XwsCStb74emERTx9v,https://api.spotify.com/v1/tracks/5P1z8XwsCStb...,https://api.spotify.com/v1/audio-analysis/5P1z...,159134,4,1
7,dangerous,0.753,0.837,5,-4.673,0,0.2780,0.065700,0.000000,0.151,0.589,151.949,audio_features,0fyP96TikhD9sM4eKqw5Yp,spotify:track:0fyP96TikhD9sM4eKqw5Yp,https://api.spotify.com/v1/tracks/0fyP96TikhD9...,https://api.spotify.com/v1/audio-analysis/0fyP...,265305,4,1
8,née-nah,0.681,0.758,1,-6.524,1,0.1880,0.004650,0.000000,0.555,0.331,165.409,audio_features,2yUzr8Sr6ldG8vmHhZwTnz,spotify:track:2yUzr8Sr6ldG8vmHhZwTnz,https://api.spotify.com/v1/tracks/2yUzr8Sr6ldG...,https://api.spotify.com/v1/audio-analysis/2yUz...,220584,4,1
9,see the real,0.875,0.645,7,-6.017,0,0.1630,0.381000,0.005370,0.110,0.317,96.974,audio_features,4E64eAph6AYI98ucunrGH8,spotify:track:4E64eAph6AYI98ucunrGH8,https://api.spotify.com/v1/tracks/4E64eAph6AYI...,https://api.spotify.com/v1/audio-analysis/4E64...,182039,4,1


In [52]:
# rap 3
album_name = "The Slim Shady LP"
artist_name = "Eminem"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap3 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap3['track_name'] = track_names

# Reorder columns to have track_name first
df_rap3 = df_rap3[['track_name'] + [col for col in df_rap3.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap3['genre'] = 1

# Display the DataFrame
df_rap3

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,Public Service Announcement,0.648,0.199,1,-25.565,1,0.8830,0.275000,0.000000,0.1360,0.950,149.548,audio_features,3GhcNCBIEMyrcpCJ8ccvXr,spotify:track:3GhcNCBIEMyrcpCJ8ccvXr,https://api.spotify.com/v1/tracks/3GhcNCBIEMyr...,https://api.spotify.com/v1/audio-analysis/3Ghc...,33227,1,1
1,My Name Is,0.869,0.680,1,-6.233,1,0.3180,0.041600,0.000001,0.0914,0.815,85.519,audio_features,75IN3CtuZwTHTnZvYM4qnJ,spotify:track:75IN3CtuZwTHTnZvYM4qnJ,https://api.spotify.com/v1/tracks/75IN3CtuZwTH...,https://api.spotify.com/v1/audio-analysis/75IN...,268400,4,1
2,Guilty Conscience,0.660,0.789,7,-5.143,1,0.3640,0.052700,0.000000,0.3410,0.513,182.218,audio_features,6da04YB906XonC6IQNC44u,spotify:track:6da04YB906XonC6IQNC44u,https://api.spotify.com/v1/tracks/6da04YB906Xo...,https://api.spotify.com/v1/audio-analysis/6da0...,199333,4,1
3,Brain Damage,0.866,0.599,2,-7.903,1,0.1630,0.009170,0.000000,0.0655,0.831,94.982,audio_features,099MPsZhn5HkEUNlpA89rK,spotify:track:099MPsZhn5HkEUNlpA89rK,https://api.spotify.com/v1/tracks/099MPsZhn5Hk...,https://api.spotify.com/v1/audio-analysis/099M...,226733,4,1
4,Paul - Skit,0.816,0.266,8,-19.654,1,0.8650,0.981000,0.000059,0.7990,0.837,130.845,audio_features,0AaKi10VdO06R4fkBFBslc,spotify:track:0AaKi10VdO06R4fkBFBslc,https://api.spotify.com/v1/tracks/0AaKi10VdO06...,https://api.spotify.com/v1/audio-analysis/0AaK...,15840,4,1
5,If I Had,0.791,0.601,7,-4.978,0,0.2170,0.000627,0.000031,0.1070,0.272,168.161,audio_features,4Q7UkPYq3NGTwhAMjfeE4a,spotify:track:4Q7UkPYq3NGTwhAMjfeE4a,https://api.spotify.com/v1/tracks/4Q7UkPYq3NGT...,https://api.spotify.com/v1/audio-analysis/4Q7U...,245493,3,1
6,'97 Bonnie & Clyde,0.713,0.909,9,-6.535,0,0.5270,0.094000,0.000169,0.3060,0.730,184.032,audio_features,6PJiaxg1lFSJFdMJrW7avo,spotify:track:6PJiaxg1lFSJFdMJrW7avo,https://api.spotify.com/v1/tracks/6PJiaxg1lFSJ...,https://api.spotify.com/v1/audio-analysis/6PJi...,316400,4,1
7,Bitch - Skit,0.804,0.336,6,-16.042,1,0.8820,0.991000,0.003470,0.6250,0.968,97.607,audio_features,5OsBkrdyDlnDZ5siaAOtVt,spotify:track:5OsBkrdyDlnDZ5siaAOtVt,https://api.spotify.com/v1/tracks/5OsBkrdyDlnD...,https://api.spotify.com/v1/audio-analysis/5OsB...,19707,3,1
8,Role Model,0.865,0.589,7,-4.224,1,0.2580,0.021800,0.000005,0.0400,0.776,91.968,audio_features,0EtirusGt3LivUmALXSMVe,spotify:track:0EtirusGt3LivUmALXSMVe,https://api.spotify.com/v1/tracks/0EtirusGt3Li...,https://api.spotify.com/v1/audio-analysis/0Eti...,205400,4,1
9,Lounge - Skit,0.303,0.376,4,-18.938,1,0.1170,0.570000,0.000000,0.8860,0.796,131.007,audio_features,1LoEersiNbdCJv96e8KjNk,spotify:track:1LoEersiNbdCJv96e8KjNk,https://api.spotify.com/v1/tracks/1LoEersiNbdC...,https://api.spotify.com/v1/audio-analysis/1LoE...,46533,1,1


In [53]:
# rap 3
album_name = "To Pimp a Butterfly"
artist_name = "Kendrick Lamar"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap4 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap4['track_name'] = track_names

# Reorder columns to have track_name first
df_rap4 = df_rap4[['track_name'] + [col for col in df_rap4.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap4['genre'] = 1

# Display the DataFrame
df_rap4

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,Wesley's Theory,0.509,0.787,7,-6.754,1,0.2650,0.35900,0.000028,0.2600,0.507,114.449,audio_features,7Ks4VCY1wFebnOdJrM13t6,spotify:track:7Ks4VCY1wFebnOdJrM13t6,https://api.spotify.com/v1/tracks/7Ks4VCY1wFeb...,https://api.spotify.com/v1/audio-analysis/7Ks4...,287360,4,1
1,For Free? - Interlude,0.526,0.896,8,-4.708,1,0.1970,0.58800,0.000000,0.3700,0.745,129.313,audio_features,75izAeHlGuOgA865CFDyIv,spotify:track:75izAeHlGuOgA865CFDyIv,https://api.spotify.com/v1/tracks/75izAeHlGuOg...,https://api.spotify.com/v1/audio-analysis/75iz...,130893,1,1
2,King Kunta,0.884,0.657,6,-4.569,0,0.0977,0.00589,0.000000,0.1540,0.489,107.059,audio_features,0N3W5peJUQtI4eyR6GJT5O,spotify:track:0N3W5peJUQtI4eyR6GJT5O,https://api.spotify.com/v1/tracks/0N3W5peJUQtI...,https://api.spotify.com/v1/audio-analysis/0N3W...,234693,4,1
3,Institutionalized,0.539,0.731,3,-5.085,0,0.4530,0.11200,0.000001,0.1290,0.568,81.114,audio_features,4oFtLSgHyZPNYDCcANhTnO,spotify:track:4oFtLSgHyZPNYDCcANhTnO,https://api.spotify.com/v1/tracks/4oFtLSgHyZPN...,https://api.spotify.com/v1/audio-analysis/4oFt...,271773,4,1
4,These Walls,0.752,0.489,1,-5.233,1,0.1370,0.11000,0.000004,0.2020,0.373,108.005,audio_features,7uWVT3UkCAZyANvv0bdyQn,spotify:track:7uWVT3UkCAZyANvv0bdyQn,https://api.spotify.com/v1/tracks/7uWVT3UkCAZy...,https://api.spotify.com/v1/audio-analysis/7uWV...,300973,4,1
5,u,0.406,0.798,11,-5.585,0,0.4320,0.26000,0.000000,0.4610,0.580,71.547,audio_features,1bxEpNR75Hq3T2oF9AZjt8,spotify:track:1bxEpNR75Hq3T2oF9AZjt8,https://api.spotify.com/v1/tracks/1bxEpNR75Hq3...,https://api.spotify.com/v1/audio-analysis/1bxE...,268493,4,1
6,Alright,0.796,0.766,7,-5.974,1,0.2380,0.07420,0.000000,0.0827,0.558,110.034,audio_features,3iVcZ5G6tvkXZkZKlMpIUs,spotify:track:3iVcZ5G6tvkXZkZKlMpIUs,https://api.spotify.com/v1/tracks/3iVcZ5G6tvkX...,https://api.spotify.com/v1/audio-analysis/3iVc...,219333,4,1
7,For Sale? - Interlude,0.301,0.686,2,-8.360,1,0.0831,0.44200,0.000115,0.1210,0.276,79.036,audio_features,7unx2Rvsq8D0RfOLRXclTA,spotify:track:7unx2Rvsq8D0RfOLRXclTA,https://api.spotify.com/v1/tracks/7unx2Rvsq8D0...,https://api.spotify.com/v1/audio-analysis/7unx...,291720,3,1
8,Momma,0.615,0.743,1,-4.580,1,0.3250,0.25400,0.000000,0.6650,0.515,111.914,audio_features,0CjJqkWOpIUXdem5hUcxAk,spotify:track:0CjJqkWOpIUXdem5hUcxAk,https://api.spotify.com/v1/tracks/0CjJqkWOpIUX...,https://api.spotify.com/v1/audio-analysis/0CjJ...,283120,5,1
9,Hood Politics,0.731,0.661,6,-6.562,0,0.3760,0.14500,0.000000,0.1200,0.369,87.912,audio_features,2UOYzhusMTypF7oAQwksCj,spotify:track:2UOYzhusMTypF7oAQwksCj,https://api.spotify.com/v1/tracks/2UOYzhusMTyp...,https://api.spotify.com/v1/audio-analysis/2UOY...,292520,4,1


In [54]:
# rap 4
album_name = "UTOPIA"
artist_name = "Travis Scott"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap5 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap5['track_name'] = track_names

# Reorder columns to have track_name first
df_rap5 = df_rap5[['track_name'] + [col for col in df_rap5.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap5['genre'] = 1

# Display the DataFrame
df_rap5

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,HYAENA,0.608,0.754,8,-5.277,1,0.0719,0.030500,0.000000,0.1190,0.455,97.071,audio_features,0hL9gOw6XBWsygEUcVjxEc,spotify:track:0hL9gOw6XBWsygEUcVjxEc,https://api.spotify.com/v1/tracks/0hL9gOw6XBWs...,https://api.spotify.com/v1/audio-analysis/0hL9...,222085,4,1
1,THANK GOD,0.657,0.553,10,-7.080,0,0.0394,0.254000,0.000000,0.1290,0.576,134.925,audio_features,1PH2MDbcBAU094DlgTIND1,spotify:track:1PH2MDbcBAU094DlgTIND1,https://api.spotify.com/v1/tracks/1PH2MDbcBAU0...,https://api.spotify.com/v1/audio-analysis/1PH2...,184889,4,1
2,MODERN JAM (feat. Teezo Touchdown),0.517,0.539,9,-6.474,0,0.0380,0.000564,0.000014,0.5370,0.531,108.984,audio_features,3KCNiDi9Pza6ZD8FggNoaw,spotify:track:3KCNiDi9Pza6ZD8FggNoaw,https://api.spotify.com/v1/tracks/3KCNiDi9Pza6...,https://api.spotify.com/v1/audio-analysis/3KCN...,255413,4,1
3,MY EYES,0.455,0.621,0,-7.694,0,0.0517,0.651000,0.000003,0.1230,0.280,119.043,audio_features,4kjI1gwQZRKNDkw1nI475M,spotify:track:4kjI1gwQZRKNDkw1nI475M,https://api.spotify.com/v1/tracks/4kjI1gwQZRKN...,https://api.spotify.com/v1/audio-analysis/4kjI...,251250,4,1
4,GOD'S COUNTRY,0.624,0.650,2,-5.819,1,0.0358,0.022900,0.000012,0.1450,0.138,150.048,audio_features,04WxWo7XeVyx22xEsrWRUb,spotify:track:04WxWo7XeVyx22xEsrWRUb,https://api.spotify.com/v1/tracks/04WxWo7XeVyx...,https://api.spotify.com/v1/audio-analysis/04Wx...,127992,4,1
5,SIRENS,0.588,0.880,9,-6.117,0,0.0747,0.097600,0.000167,0.7670,0.242,96.003,audio_features,7LSpFCvRZZot2AlmkUzy9k,spotify:track:7LSpFCvRZZot2AlmkUzy9k,https://api.spotify.com/v1/tracks/7LSpFCvRZZot...,https://api.spotify.com/v1/audio-analysis/7LSp...,204447,3,1
6,MELTDOWN (feat. Drake),0.557,0.774,7,-5.275,0,0.3510,0.012000,0.000000,0.3960,0.397,111.975,audio_features,67nepsnrcZkowTxMWigSbb,spotify:track:67nepsnrcZkowTxMWigSbb,https://api.spotify.com/v1/tracks/67nepsnrcZko...,https://api.spotify.com/v1/audio-analysis/67ne...,246134,4,1
7,FE!N (feat. Playboi Carti),0.569,0.882,3,-2.777,0,0.0600,0.031600,0.000000,0.1420,0.201,148.038,audio_features,42VsgItocQwOQC3XWZ8JNA,spotify:track:42VsgItocQwOQC3XWZ8JNA,https://api.spotify.com/v1/tracks/42VsgItocQwO...,https://api.spotify.com/v1/audio-analysis/42Vs...,191701,4,1
8,DELRESTO (ECHOES) (feat. Beyoncé),0.674,0.609,2,-6.306,0,0.0450,0.461000,0.000000,0.3280,0.509,116.066,audio_features,0OEe83mMZ5kaNw5uZQ7ilG,spotify:track:0OEe83mMZ5kaNw5uZQ7ilG,https://api.spotify.com/v1/tracks/0OEe83mMZ5ka...,https://api.spotify.com/v1/audio-analysis/0OEe...,274138,4,1
9,I KNOW ?,0.927,0.619,5,-4.441,0,0.0539,0.018600,0.000000,0.1040,0.817,117.995,audio_features,6wsqVwoiVH2kde4k4KKAFU,spotify:track:6wsqVwoiVH2kde4k4KKAFU,https://api.spotify.com/v1/tracks/6wsqVwoiVH2k...,https://api.spotify.com/v1/audio-analysis/6wsq...,211582,4,1


In [55]:
# rap 5
album_name = "UTOPIA"
artist_name = "Travis Scott"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap5 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap5['track_name'] = track_names

# Reorder columns to have track_name first
df_rap5 = df_rap5[['track_name'] + [col for col in df_rap5.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap5['genre'] = 1

# Display the DataFrame
df_rap5

,track_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,HYAENA,0.608,0.754,8,-5.277,1,0.0719,0.030500,0.000000,0.1190,0.455,97.071,audio_features,0hL9gOw6XBWsygEUcVjxEc,spotify:track:0hL9gOw6XBWsygEUcVjxEc,https://api.spotify.com/v1/tracks/0hL9gOw6XBWs...,https://api.spotify.com/v1/audio-analysis/0hL9...,222085,4,1
1,THANK GOD,0.657,0.553,10,-7.080,0,0.0394,0.254000,0.000000,0.1290,0.576,134.925,audio_features,1PH2MDbcBAU094DlgTIND1,spotify:track:1PH2MDbcBAU094DlgTIND1,https://api.spotify.com/v1/tracks/1PH2MDbcBAU0...,https://api.spotify.com/v1/audio-analysis/1PH2...,184889,4,1
2,MODERN JAM (feat. Teezo Touchdown),0.517,0.539,9,-6.474,0,0.0380,0.000564,0.000014,0.5370,0.531,108.984,audio_features,3KCNiDi9Pza6ZD8FggNoaw,spotify:track:3KCNiDi9Pza6ZD8FggNoaw,https://api.spotify.com/v1/tracks/3KCNiDi9Pza6...,https://api.spotify.com/v1/audio-analysis/3KCN...,255413,4,1
3,MY EYES,0.455,0.621,0,-7.694,0,0.0517,0.651000,0.000003,0.1230,0.280,119.043,audio_features,4kjI1gwQZRKNDkw1nI475M,spotify:track:4kjI1gwQZRKNDkw1nI475M,https://api.spotify.com/v1/tracks/4kjI1gwQZRKN...,https://api.spotify.com/v1/audio-analysis/4kjI...,251250,4,1
4,GOD'S COUNTRY,0.624,0.650,2,-5.819,1,0.0358,0.022900,0.000012,0.1450,0.138,150.048,audio_features,04WxWo7XeVyx22xEsrWRUb,spotify:track:04WxWo7XeVyx22xEsrWRUb,https://api.spotify.com/v1/tracks/04WxWo7XeVyx...,https://api.spotify.com/v1/audio-analysis/04Wx...,127992,4,1
5,SIRENS,0.588,0.880,9,-6.117,0,0.0747,0.097600,0.000167,0.7670,0.242,96.003,audio_features,7LSpFCvRZZot2AlmkUzy9k,spotify:track:7LSpFCvRZZot2AlmkUzy9k,https://api.spotify.com/v1/tracks/7LSpFCvRZZot...,https://api.spotify.com/v1/audio-analysis/7LSp...,204447,3,1
6,MELTDOWN (feat. Drake),0.557,0.774,7,-5.275,0,0.3510,0.012000,0.000000,0.3960,0.397,111.975,audio_features,67nepsnrcZkowTxMWigSbb,spotify:track:67nepsnrcZkowTxMWigSbb,https://api.spotify.com/v1/tracks/67nepsnrcZko...,https://api.spotify.com/v1/audio-analysis/67ne...,246134,4,1
7,FE!N (feat. Playboi Carti),0.569,0.882,3,-2.777,0,0.0600,0.031600,0.000000,0.1420,0.201,148.038,audio_features,42VsgItocQwOQC3XWZ8JNA,spotify:track:42VsgItocQwOQC3XWZ8JNA,https://api.spotify.com/v1/tracks/42VsgItocQwO...,https://api.spotify.com/v1/audio-analysis/42Vs...,191701,4,1
8,DELRESTO (ECHOES) (feat. Beyoncé),0.674,0.609,2,-6.306,0,0.0450,0.461000,0.000000,0.3280,0.509,116.066,audio_features,0OEe83mMZ5kaNw5uZQ7ilG,spotify:track:0OEe83mMZ5kaNw5uZQ7ilG,https://api.spotify.com/v1/tracks/0OEe83mMZ5ka...,https://api.spotify.com/v1/audio-analysis/0OEe...,274138,4,1
9,I KNOW ?,0.927,0.619,5,-4.441,0,0.0539,0.018600,0.000000,0.1040,0.817,117.995,audio_features,6wsqVwoiVH2kde4k4KKAFU,spotify:track:6wsqVwoiVH2kde4k4KKAFU,https://api.spotify.com/v1/tracks/6wsqVwoiVH2k...,https://api.spotify.com/v1/audio-analysis/6wsq...,211582,4,1


In [ ]:
# rap 5
album_name = "UTOPIA"
artist_name = "Travis Scott"
results = sp.search(q=f'album:{album_name} artist:{artist_name}', type='album')
album_id = results['albums']['items'][0]['id']

# Get album details using the album ID
album = sp.album(album_id)

# Get the list of track IDs from the album
track_ids = [track['id'] for track in album['tracks']['items']]

# Fetch audio features for each track
audio_features = sp.audio_features(track_ids)

# Filter out None values from audio features
audio_features = [feature for feature in audio_features if feature is not None]

# Create a DataFrame to store the audio features
df_rap5 = pd.DataFrame(audio_features)

# Add track names to the DataFrame for better readability
track_names = [track['name'] for track in album['tracks']['items'] if track['id'] in [feature['id'] for feature in audio_features]]
df_rap5['track_name'] = track_names

# Reorder columns to have track_name first
df_rap5 = df_rap5[['track_name'] + [col for col in df_rap5.columns if col != 'track_name']]

# Add a new column 'genre' with all values set to 1
df_rap5['genre'] = 1

# Display the DataFrame
df_rap5